# 1. Data Capture / Mediapipe setup

In [ ]:
# Cell 1: Install libraries
!pip install mediapipe opencv-python pandas scikit-learn numpy matplotlib

In [1]:
# Cell 2: Libraries
import mediapipe as mp
import cv2
import numpy as np
import time
import pandas as pd
import csv
import os
from matplotlib import pyplot as plt

In [ ]:
# Cell 3: Check that the camera and mediapipe are working
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose, \
        mp.solutions.hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            ret, image = cap.read()
            if not ret:
                print("Error: Unable to read frame from video capture")
                break

            # Flip image to simulate mirror view
            image = cv2.flip(image, 1)

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detections
            pose_results = pose.process(image)
            hand_results = hands.process(image)

            # RGB 2 BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Draw points
            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
                )

            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp.solutions.hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                        mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                    )

            cv2.imshow("Raw Webcam Feed", image)

            if cv2.waitKey(10) & 0xFF == ord("q"):
                break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# Cell 4: Record video for data capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30 
    print(f"Video Resolution: {width}x{height} at {fps} FPS")

    fourcc = cv2.VideoWriter_fourcc(*'XVID') 
    out = cv2.VideoWriter('output_2.avi', fourcc, fps, (width, height))

    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Error: Unable to read frame from video capture")
            break
        frame = cv2.flip(frame, 1)

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_rgb.flags.writeable = False
        results = hands.process(frame_rgb)  
        frame_rgb.flags.writeable = True
        frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

        # Write every frame
        out.write(frame)

        cv2.imshow('MediaPipe Hands', frame)  
        if cv2.waitKey(5) & 0xFF == ord('q'): 
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [4]:
# Cell 5: Create CSV to store data
headers = ['class', 'accuracy', 'sequence'] 
headers.extend([f'pose_{coord}{i}' for i in range(33) for coord in ('x', 'y', 'z', 'v')])
headers.extend([f'{hand}_{coord}{i}' for hand in ('right_hand', 'left_hand') for i in range(21) for coord in ('x', 'y', 'z', 'v')])

with open('coordinates_1.csv', mode='w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(headers)

In [5]:
# Cell 6: Capture data for first movement
cap = cv2.VideoCapture('output_1.avi')
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('processed_output.avi', fourcc, fps, (width, height))

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils
    pose_drawing_spec = mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4)
    hand_drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)

    record = False
    accuracy = None
    sequences = {'r': -1, 'w': -1}  
    recording_state = None

    with open('coordinates_1.csv', mode='a', newline='') as file:
        csv_writer = csv.writer(file)

        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Error: Unable to read frame from video capture")
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_rgb.flags.writeable = False
            pose_results = pose.process(frame_rgb)
            hand_results = hands.process(frame_rgb)
            frame_rgb.flags.writeable = True
            frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=pose_drawing_spec,
                    connection_drawing_spec=pose_drawing_spec)

            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        landmark_drawing_spec=hand_drawing_spec,
                        connection_drawing_spec=hand_drawing_spec)

            # Select the sequence number based on the key pressed
            current_sequence = sequences[recording_state] if recording_state else -1
            row = ['', accuracy, current_sequence]

            if pose_results.pose_landmarks:
                for lm in pose_results.pose_landmarks.landmark:
                    visibility_binary = 1 if lm.visibility > 0.3 else 0
                    row.extend([lm.x, lm.y, lm.z, visibility_binary])
            else:
                row.extend([0] * 33 * 4)

            for hand in ('right_hand', 'left_hand'):
                found = False
                if hand_results.multi_hand_landmarks:
                    for hand_landmarks, handedness in zip(hand_results.multi_hand_landmarks, hand_results.multi_handedness):
                        if handedness.classification[0].label == ('Right' if hand == 'right_hand' else 'Left'):
                            for lm in hand_landmarks.landmark:
                                visibility_binary = 1 if lm.visibility > 0.2 else 0
                                row.extend([lm.x, lm.y, lm.z, visibility_binary])
                            found = True
                            break
                if not found:
                    row.extend([0] * 21 * 4)

            if record:
                csv_writer.writerow(row)

            out.write(frame)
            cv2.imshow('MediaPipe Pose', frame)
            key = cv2.waitKey(5) & 0xFF

            if key == ord('r') or key == ord('w'):
                new_state = chr(key)
                if new_state != recording_state:
                    sequences[new_state] += 1  
                    recording_state = new_state
                record = True
                accuracy = 1 if key == ord('r') else 0
            elif key == ord('s'):
                record = False
                recording_state = None
            elif key == ord('q'):
                break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

df = pd.read_csv('coordinates_1.csv')
df.loc[df['accuracy'].notna(), 'class'] = 'both_hands'
df.to_csv('coordinates_1.csv', index=False)


I0000 00:00:1715673906.080072   93500 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro
I0000 00:00:1715673906.084599   93500 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro
W0000 00:00:1715673906.089316   95225 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1715673906.094879   95225 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1715673906.176681   95210 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1715673906.183972   95210 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/jorgemuyo/Desktop/ML_Craftsmanship/tf

Error: Unable to read frame from video capture


/var/folders/wv/_x9hjmys03x5gnbfl70ry2sr0000gn/T/ipykernel_5699/3491012288.py:108: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'both_hands' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['accuracy'].notna(), 'class'] = 'both_hands'


In [6]:
# Cell 7: Capture data for second movement
cap = cv2.VideoCapture('output_2.avi')
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('processed_output.avi', fourcc, fps, (width, height))

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils
    pose_drawing_spec = mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4)
    hand_drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)

    record = False
    accuracy = None
    sequences = {'r': -1, 'w': -1}  
    recording_state = None

    with open('coordinates_2.csv', mode='a', newline='') as file:
        csv_writer = csv.writer(file, lineterminator='\n')

        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Error: Unable to read frame from video capture")
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_rgb.flags.writeable = False
            pose_results = pose.process(frame_rgb)
            hand_results = hands.process(frame_rgb)
            frame_rgb.flags.writeable = True
            frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=pose_drawing_spec,
                    connection_drawing_spec=pose_drawing_spec)

            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        landmark_drawing_spec=hand_drawing_spec,
                        connection_drawing_spec=hand_drawing_spec)

            current_sequence = sequences[recording_state] if recording_state else -1
            row = ['', accuracy, current_sequence]

            if pose_results.pose_landmarks:
                for lm in pose_results.pose_landmarks.landmark:
                    visibility_binary = 1 if lm.visibility > 0.3 else 0
                    row.extend([lm.x, lm.y, lm.z, visibility_binary])
            else:
                row.extend([0] * 33 * 4)

            for hand in ('right_hand', 'left_hand'):
                found = False
                if hand_results.multi_hand_landmarks:
                    for hand_landmarks, handedness in zip(hand_results.multi_hand_landmarks, hand_results.multi_handedness):
                        if handedness.classification[0].label == ('Right' if hand == 'right_hand' else 'Left'):
                            for lm in hand_landmarks.landmark:
                                visibility_binary = 1 if lm.visibility > 0.2 else 0
                                row.extend([lm.x, lm.y, lm.z, visibility_binary])
                            found = True
                            break
                if not found:
                    row.extend([0] * 21 * 4)

            if record:
                csv_writer.writerow(row)

            out.write(frame)
            cv2.imshow('MediaPipe Pose', frame)
            key = cv2.waitKey(5) & 0xFF

            if key == ord('r') or key == ord('w'):
                new_state = chr(key)
                if new_state != recording_state:
                    sequences[new_state] += 1  
                    recording_state = new_state
                record = True
                accuracy = 1 if key == ord('r') else 0
            elif key == ord('s'):
                record = False
                recording_state = None
            elif key == ord('q'):
                break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

df = pd.read_csv('coordinates_2.csv', header=None)
df.loc[df[1].notna(), 0] = 'one_hand'
df.to_csv('coordinates_2.csv', header=False, index=False)


I0000 00:00:1715674121.492008   93500 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro
I0000 00:00:1715674121.496735   93500 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro
W0000 00:00:1715674121.504388   97021 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1715674121.511112   97019 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1715674121.590845   97009 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1715674121.598340   97008 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/jorgemuyo/Desktop/ML_Craftsmanship/tf

In [ ]:
# Cell 7.2: Capture data for third movement
cap = cv2.VideoCapture('output_scissors.avi')
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('processed_output.avi', fourcc, fps, (width, height))

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils
    pose_drawing_spec = mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4)
    hand_drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)

    record = False
    accuracy = None
    sequences = {'r': -1, 'w': -1}  
    recording_state = None

    with open('coordinates_3.csv', mode='a', newline='') as file:
        csv_writer = csv.writer(file, lineterminator='\n')

        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Error: Unable to read frame from video capture")
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_rgb.flags.writeable = False
            pose_results = pose.process(frame_rgb)
            hand_results = hands.process(frame_rgb)
            frame_rgb.flags.writeable = True
            frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=pose_drawing_spec,
                    connection_drawing_spec=pose_drawing_spec)

            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        landmark_drawing_spec=hand_drawing_spec,
                        connection_drawing_spec=hand_drawing_spec)

            current_sequence = sequences[recording_state] if recording_state else -1
            row = ['', accuracy, current_sequence]

            if pose_results.pose_landmarks:
                for lm in pose_results.pose_landmarks.landmark:
                    visibility_binary = 1 if lm.visibility > 0.3 else 0
                    row.extend([lm.x, lm.y, lm.z, visibility_binary])
            else:
                row.extend([0] * 33 * 4)

            for hand in ('right_hand', 'left_hand'):
                found = False
                if hand_results.multi_hand_landmarks:
                    for hand_landmarks, handedness in zip(hand_results.multi_hand_landmarks, hand_results.multi_handedness):
                        if handedness.classification[0].label == ('Right' if hand == 'right_hand' else 'Left'):
                            for lm in hand_landmarks.landmark:
                                visibility_binary = 1 if lm.visibility > 0.2 else 0
                                row.extend([lm.x, lm.y, lm.z, visibility_binary])
                            found = True
                            break
                if not found:
                    row.extend([0] * 21 * 4)

            if record:
                csv_writer.writerow(row)

            out.write(frame)
            cv2.imshow('MediaPipe Pose', frame)
            key = cv2.waitKey(5) & 0xFF

            if key == ord('r') or key == ord('w'):
                new_state = chr(key)
                if new_state != recording_state:
                    sequences[new_state] += 1  
                    recording_state = new_state
                record = True
                accuracy = 1 if key == ord('r') else 0
            elif key == ord('s'):
                record = False
                recording_state = None
            elif key == ord('q'):
                break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

df = pd.read_csv('coordinates_3.csv', header=None)
df.loc[df[1].notna(), 0] = ''
df.to_csv('coordinates_3.csv', header=False, index=False)


# 2. Data Processing

In [7]:
# Cell 8: Join all the CSVs of the different movements
df1 = pd.read_csv('coordinates_1.csv', header=None)
df2 = pd.read_csv('coordinates_2.csv', header=None)
# df3 = pd.read_csv('coordinates_3.csv', header=None)


combined_df = pd.concat([df1, df2], axis=0, ignore_index=True)
combined_path = 'combined_coordinates.csv'
combined_df.to_csv(combined_path, header=False, index=False)

In [16]:
# Cell 9: Delete not important data
data = pd.read_csv('combined_coordinates.csv')

# Identify columns to remove: include specific ranges and all visibility points
#columns_to_remove_1 = [f"pose_{c}{i}" for c in ['x', 'y', 'z', 'v'] for i in range(0, 11)]
#columns_to_remove_2 = [f"pose_{c}{i}" for c in ['x', 'y', 'z', 'v'] for i in range(23, 33)]
visibility_columns_pose = [col for col in data.columns if 'pose_v' in col]
visibility_columns_left_hand = [col for col in data.columns if 'left_hand_v' in col]
visibility_columns_right_hand = [col for col in data.columns if 'right_hand_v' in col]

# Combine all columns to remove
columns_to_remove = visibility_columns_pose + visibility_columns_left_hand + visibility_columns_right_hand
columns_to_remove = [col for col in columns_to_remove if col in data.columns]

# Drop the selected columns from the dataframe
data_filtered = data.drop(columns=columns_to_remove)

data_filtered.to_csv('filtered_coordinates.csv', index=False)

# Display the first few rows of the filtered dataset
data_filtered.head()

,class,accuracy,sequence,pose_x0,pose_y0,pose_z0,pose_x1,pose_y1,pose_z1,pose_x2,...,left_hand_z17,left_hand_x18,left_hand_y18,left_hand_z18,left_hand_x19,left_hand_y19,left_hand_z19,left_hand_x20,left_hand_y20,left_hand_z20
0,both_hands,1,0,0.478681,0.384277,-0.750511,0.500309,0.323448,-0.696990,0.515079,...,-0.013819,0.161316,0.221783,-0.020189,0.161720,0.185594,-0.023155,0.162091,0.153087,-0.025024
1,both_hands,1,0,0.478646,0.383587,-0.701445,0.500199,0.322867,-0.662416,0.515061,...,-0.016886,0.102202,0.244032,-0.023942,0.100457,0.212051,-0.026873,0.099571,0.183783,-0.028546
2,both_hands,1,0,0.478682,0.383573,-0.676765,0.500229,0.322869,-0.640812,0.515144,...,-0.014247,0.059026,0.245731,-0.022560,0.056664,0.207527,-0.026945,0.055423,0.172296,-0.029401
3,both_hands,1,0,0.478991,0.383722,-0.671898,0.500861,0.322870,-0.633345,0.515918,...,-0.023007,0.038607,0.258966,-0.032427,0.036734,0.215381,-0.036107,0.036500,0.178063,-0.037599
4,both_hands,1,0,0.479233,0.384098,-0.670311,0.501366,0.322927,-0.632044,0.516547,...,-0.016815,0.040668,0.258800,-0.025032,0.037678,0.217694,-0.028538,0.036706,0.181148,-0.030353


In [17]:
# Cell 10: Re-label CSV
def transform_and_merge_columns(file_path, output_file_path):
    # Load the CSV file
    data = pd.read_csv(file_path)
    
    data['accuracy'] = data['accuracy'].map({0: 'W', 1: 'R'})
    data['label'] = data.iloc[:, 0].astype(str) + '_' + data['accuracy']
    
    label_column = data.pop('label') 
    data.insert(0, 'label', label_column)
    
    data.drop(columns=['class', 'accuracy'], inplace=True)
    data.to_csv(output_file_path, index=False)

input_file_path = 'filtered_coordinates.csv'  
output_file_path = 'modified_file.csv'  
transform_and_merge_columns(input_file_path, output_file_path)

In [18]:
# Cell 11: Knowing the size of the CSV
data = pd.read_csv('modified_file.csv')

rows, columns = data.shape
print("Rows:", rows)
print("Columns:", columns) # We will take out two columns from here to exclude the labeling
21*3+21*3+33*3 # Make sure the points you want to collect are the same. amount as columns you have (lh_lm*3 + rh_lm*3 + pose_lm*3)

Rows: 2001
Columns: 227


225

In [70]:
# Cell 12: Capture the different labels of the dataset
labels = data.iloc[:, 0]
actions = list(set(labels))
print(actions)

['both_hands_W', 'both_hands_R', 'one_hand_R', 'one_hand_W']


In [20]:
# Cell 13: Create directory to save Data arrays for training
def create_subfolders_from_labels_and_sequences(csv_file, base_folder):
    # Load the data
    data = pd.read_csv(csv_file)
    
    if not os.path.exists(base_folder): 
        os.makedirs(base_folder)
    
    unique_labels = data['label'].unique()     
    
    for label in unique_labels:    
        label_folder_path = os.path.join(base_folder, label)
        if not os.path.exists(label_folder_path):
            os.makedirs(label_folder_path)
        
        label_data = data[data['label'] == label]       
        
        unique_sequences = label_data['sequence'].unique()
        
        for sequence in unique_sequences:
            sequence_folder_path = os.path.join(label_folder_path, str(sequence))
            if not os.path.exists(sequence_folder_path):
                os.makedirs(sequence_folder_path)
            else:
                print(f"Subfolder for sequence {sequence} in label {label} already exists.")

file_path = 'modified_file.csv'
DATA_PATH = os.path.join('DataBase')
create_subfolders_from_labels_and_sequences(file_path, DATA_PATH)

In [21]:
# Cell 14: Save arrays in directory
def save_data_as_arrays(csv_file, base_folder):
    # Load the data
    data = pd.read_csv(csv_file)
    
    for (label, sequence), group in data.groupby(['label', 'sequence']):
        folder_path = os.path.join(base_folder, label, str(sequence))
        
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        
        file_counter = 0
        
        for index, row in group.iterrows():
            data_array = row.drop(['label', 'sequence']).to_numpy(dtype=np.float32)
            
            file_name = f"{file_counter}.npy"
            file_path = os.path.join(folder_path, file_name)
            
            np.save(file_path, data_array)
            file_counter += 1

save_data_as_arrays(file_path, DATA_PATH)

In [28]:
# Cell 15: Test new arrays are created in proper format (x, )
test = np.load('DataBase/one_hand_R/1/14.npy', allow_pickle=True)
test.shape

(225,)

In [23]:
# Cell 16: Create new amount of folders so all the sequences and movements have the same amount of data
def equalize_subfolders(base_folder):
    subfolder_counts = {}

    labels = [d for d in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, d))]
    for label in labels:
        label_path = os.path.join(base_folder, label)
        sequences = [d for d in os.listdir(label_path) if os.path.isdir(os.path.join(label_path, d))]
        subfolder_counts[label] = len(sequences)
    
    max_subfolders = max(subfolder_counts.values())
    
    for label, count in subfolder_counts.items():
        label_path = os.path.join(base_folder, label)
        if count < max_subfolders:
            for i in range(count, max_subfolders):
                new_folder_path = os.path.join(label_path, str(i))
                if not os.path.exists(new_folder_path):
                    os.makedirs(new_folder_path)
                print(f"Created folder {new_folder_path}")

equalize_subfolders(DATA_PATH)

Created folder DataBase/one_hand_R/35
Created folder DataBase/one_hand_R/36
Created folder DataBase/both_hands_W/30
Created folder DataBase/both_hands_W/31
Created folder DataBase/both_hands_W/32
Created folder DataBase/both_hands_W/33
Created folder DataBase/both_hands_W/34
Created folder DataBase/both_hands_W/35
Created folder DataBase/both_hands_W/36
Created folder DataBase/both_hands_R/33
Created folder DataBase/both_hands_R/34
Created folder DataBase/both_hands_R/35
Created folder DataBase/both_hands_R/36


In [24]:
# Cell 17: Create np.arrays filled with 0s to have the same amount of data per movement and sequence
def equalize_array_counts(base_folder):
    max_files = 0

    # Dictionary to hold all subfolder paths
    subfolder_paths = []

    # Traverse through all label and sequence folders
    for label in os.listdir(base_folder):
        label_path = os.path.join(base_folder, label)
        if os.path.isdir(label_path):
            for sequence in os.listdir(label_path):
                sequence_path = os.path.join(label_path, sequence)
                if os.path.isdir(sequence_path):
                    # Collect all subfolder paths
                    subfolder_paths.append(sequence_path)
                    # Count the number of files in each subfolder
                    num_files = len([f for f in os.listdir(sequence_path) if f.endswith('.npy')])
                    if num_files > max_files:
                        max_files = num_files

    # Now, ensure all subfolders have the same number of files
    for folder in subfolder_paths:
        current_count = len([f for f in os.listdir(folder) if f.endswith('.npy')])
        if current_count < max_files:
            for i in range(current_count, max_files):
                new_file_path = os.path.join(folder, f"{i}.npy")
                # Create a numpy array filled with zeros
                zero_array = np.zeros(225, dtype=np.float32)  # Specify the dtype here
                np.save(new_file_path, zero_array)
                print(f"Created {new_file_path} with zeros")

# Call the function with your base folder path
equalize_array_counts(DATA_PATH)

Created DataBase/one_hand_R/32/15.npy with zeros
Created DataBase/one_hand_R/32/16.npy with zeros
Created DataBase/one_hand_R/32/17.npy with zeros
Created DataBase/one_hand_R/32/18.npy with zeros
Created DataBase/one_hand_R/32/19.npy with zeros
Created DataBase/one_hand_R/32/20.npy with zeros
Created DataBase/one_hand_R/32/21.npy with zeros
Created DataBase/one_hand_R/35/0.npy with zeros
Created DataBase/one_hand_R/35/1.npy with zeros
Created DataBase/one_hand_R/35/2.npy with zeros
Created DataBase/one_hand_R/35/3.npy with zeros
Created DataBase/one_hand_R/35/4.npy with zeros
Created DataBase/one_hand_R/35/5.npy with zeros
Created DataBase/one_hand_R/35/6.npy with zeros
Created DataBase/one_hand_R/35/7.npy with zeros
Created DataBase/one_hand_R/35/8.npy with zeros
Created DataBase/one_hand_R/35/9.npy with zeros
Created DataBase/one_hand_R/35/10.npy with zeros
Created DataBase/one_hand_R/35/11.npy with zeros
Created DataBase/one_hand_R/35/12.npy with zeros
Created DataBase/one_hand_R/35

In [29]:
# Cell 18: Re-test new arrays
test = np.load('DataBase/one_hand_W/24/15.npy')
test.shape

(225,)

In [30]:
# Cell 19: count the amount of subfolders and arrays
def count_subfolders_and_arrays(base_folder):
    # Find all label folders and sort them to consistently pick the first
    label_folders = sorted([d for d in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, d))])

    if not label_folders:
        print("No folders found in the base directory.")
        return None

    first_label_path = os.path.join(base_folder, label_folders[0])
    
    subfolders = sorted([d for d in os.listdir(first_label_path) if os.path.isdir(os.path.join(first_label_path, d))])   
    no_sequences = len(subfolders)

    if not subfolders:
        print("No subfolders found in the first label folder.")
        return None
    first_subfolder_path = os.path.join(first_label_path, subfolders[0])

    sequence_length = len([f for f in os.listdir(first_subfolder_path) if f.endswith('.npy')])

    return no_sequences, sequence_length

results = count_subfolders_and_arrays(DATA_PATH)

if results:
    no_sequences, sequence_length = results
    print(f"no_sequences= {no_sequences} / sequence_length= {sequence_length}")

no_sequences= 37 / sequence_length= 22


In [72]:
# Cell 20: Create labelmap to asign every action
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'both_hands_W': 0, 'both_hands_R': 1, 'one_hand_R': 2, 'one_hand_W': 3}

In [33]:
# Cell 21: Assign sequences and labels list
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            file_path = os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy")
            res = np.load(file_path, allow_pickle=True)  
            window.append(res)
        sequences.append(window)  
        labels.append(label_map[action])

# 3. Set up Tensorflow (Apple sillicon step if you use nviadia GPUs check [Tensorflow GPU Documentation](https://www.tensorflow.org/guide/gpu))

In [34]:
# Cell 22: Install new libraries for processing and training
!pip install tensorflow 
!pip install tensorflow-macos
!pip install tensorflow-metal
!pip install numpy --upgrade
!pip install pandas --upgrade
!pip install matplotlib --upgrade
!pip install scikit-learn --upgrade
!pip install scipy --upgrade
!pip install plotly --upgrade

In [35]:
# Cell 23: Conda environment dependencies installation (I only managed to get the Tensorflow GPU working under a conda environment)
!conda install -y apple tensorflow-deps
!conda install notebook -y

Retrieving notices: ...working... done
Channels:
 - conda-forge
Platform: osx-arm64
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - tensorflow-deps
  - apple

Current channels:

  - https://conda.anaconda.org/conda-forge

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


Channels:
 - conda-forge
Platform: osx-arm64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 24.4.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [36]:
# Cell 24: Check if your GPU works (code from fotiecodes: (https://blog.fotiecodes.com/install-tensorflow-on-your-mac-m1m2m3-with-gpu-support-clqs92bzl000308l8a3i35479))
import sys
import keras
import pandas as pd
import sklearn as sk
import scipy as sp
import tensorflow as tf
import platform

print (f"Python Platform: {platform.platform ()}")
print (f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print ()

print (f"Python {sys.version}")
print (f"Pandas {pd.__version__}")
print (f"Scikit-Learn {sk.__version__}")
print (f"SciPy {sp.__version__}")
gpu = len (tf.config.list_physical_devices ('GPU'))>0
print ("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: macOS-14.4.1-arm64-arm-64bit
Tensor Flow Version: 2.16.1
Keras Version: 3.3.3

Python 3.10.13 | packaged by conda-forge | (main, Dec 23 2023, 15:35:25) [Clang 16.0.6 ]
Pandas 2.2.2
Scikit-Learn 1.4.2
SciPy 1.13.0
GPU is available


# 4. Training

In [44]:
# Cell 25: Import training dependencies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [40]:
# Cell 26: Set X and y
X = np.array(sequences)
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1,

In [43]:
# Cell 27: Check size of array
np.array(sequences).shape

(148, 22, 225)

In [45]:
# Cell 28: Split values for testing and training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
y_train.shape

(140, 4)

In [46]:
# Cell 29: To web monitor live training later
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [51]:
# Cell 30: Build the neural network (All about experimenting. This one is probably not the most efficient way)
actions
actions_array = np.array(actions)
model = Sequential()

# Adding a Bidirectional LSTM layer with input shape
model.add(Bidirectional(LSTM(64, return_sequences=True, activation='tanh'), input_shape=(68, 225)))

# Dropout for regularization
model.add(Dropout(0.2))

# Another Bidirectional LSTM layer with regularization
model.add(Bidirectional(LSTM(128, return_sequences=True, activation='tanh', kernel_regularizer=l2(0.01))))

# Additional dropout layer
model.add(Dropout(0.2))

# Final Bidirectional LSTM layer, does not return sequences
model.add(Bidirectional(LSTM(64, activation='tanh')))

# Dense layers following the final LSTM output
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions_array.shape[0], activation='softmax'))


In [52]:
# Cell 31: Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
# Cell 32: Early stoping to get the best result of the training
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)

# ModelCheckpoint to save the best model during training
# Updated to save in `.keras` format as required
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_accuracy', mode='max')

# ReduceLROnPlateau to reduce learning rate when a plateau in validation loss is detected
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

In [54]:
# Cell 33: Training
model.fit(X_train, y_train, validation_split=0.2, epochs=2000, batch_size=32,
          callbacks=[tb_callback, early_stopping, checkpoint, reduce_lr])

Epoch 1/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 793ms/step - accuracy: 0.2174 - loss: 5.3815 - val_accuracy: 0.5000 - val_loss: 4.9417 - learning_rate: 0.0010
Epoch 2/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 269ms/step - accuracy: 0.4235 - loss: 4.8324 - val_accuracy: 0.7143 - val_loss: 4.3293 - learning_rate: 0.0010
Epoch 3/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step - accuracy: 0.5955 - loss: 4.2251 - val_accuracy: 0.5714 - val_loss: 3.7677 - learning_rate: 0.0010
Epoch 4/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step - accuracy: 0.4455 - loss: 3.7134 - val_accuracy: 0.4286 - val_loss: 3.3555 - learning_rate: 0.0010
Epoch 5/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 229ms/step - accuracy: 0.5164 - loss: 3.3131 - val_accuracy: 0.4286 - val_loss: 3.0081 - learning_rate: 0.0010
Epoch 6/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 235ms/step - accuracy: 0.4787 - loss: 3.0038 - val_accuracy: 0.5714 - val_loss: 2.6840 - learning_rate: 0.0010
Epoch 7/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 262ms/step - accuracy: 0.5737 - loss: 2.6828 -

# 5. Tests

In [56]:
# Cell 34: Save model
model.save('actions.h5')

In [ ]:
# Cell 35: Load model: In case you have your own
model = load_model('actions.h5')

In [57]:
# Cell 36: Check the structure of the model
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_6 (Bidirectional) │ (None, 68, 128)        │       148,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 68, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_7 (Bidirectional) │ (None, 68, 256)        │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 68, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_8 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,759,406 (6.71 MB)

 Trainable params: 586,468 (2.24 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,172,938 (4.47 MB)

In [58]:
# Cell 37: Prediction test
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [61]:
# Cell 38: Prediction test
actions[np.argmax(res[3])]

'both_hands_W'

In [62]:
# Cell 39: Prediction test
actions[np.argmax(y_test[3])]

'both_hands_W'

# 6. Metrics

In [63]:
# Cell 40: New libraries for seeing the preformance of the model
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [64]:
# Cell 41: yhat
yhat = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


In [65]:
# Cell 42: ytrue
ytrue = np.argmax(y_test, axis=1).tolist()  
yhat = np.argmax(yhat, axis=1).tolist()

In [66]:
# Cell 43: Confusion matrix
multilabel_confusion_matrix(ytrue, yhat)

array([[[7, 0],
        [0, 1]],

       [[7, 0],
        [0, 1]],

       [[5, 0],
        [0, 3]],

       [[5, 0],
        [0, 3]]])

In [67]:
# Cell 44: Accuracy score
accuracy_score(ytrue, yhat)

1.0

# 7. Real Time Test

In [ ]:
data = pd.read_csv('modified_file.csv')

labels = data.iloc[:, 0]
actions = list(set(labels))
print(actions)

label_map = {label:num for num, label in enumerate(actions)}
label_map

In [74]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model

# Initialize MediaPipe Holistic and Drawing utilities
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Load the model for action recognition
model_path = 'actions.h5'
model = load_model(model_path)

# Define action labels and create mapping dictionaries
# Assuming label_map is defined elsewhere in your code
reverse_label_map = {value: key for key, value in label_map.items()}

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    # Draw pose, hand, and face landmarks on the image.
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        )
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
        )
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )

def extract_keypoints(results):
    # Extract keypoints from the holistic model's results
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

sequence = []
sentence = []
threshold = 0.5

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame could not be read.")
            break

        # Flip the frame horizontally
        frame = cv2.flip(frame, 1)  # Horizontal flip

        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)

        if results.pose_landmarks or results.left_hand_landmarks or results.right_hand_landmarks:
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]  # Keep the last 30 sequences

            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                action = actions[np.argmax(res)]

                # Determine color based on the action suffix
                if action.endswith("_W"):
                    color = (0, 0, 255)  # Red color
                elif action.endswith("_R"):
                    color = (0, 255, 0)  # Green color
                else:
                    color = (255, 255, 255)  # White color for other actions

                # Display the predicted action on the screen
                cv2.putText(image, f'Action: {action}', (15, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


I0000 00:00:1715695113.485040   93500 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro
W0000 00:00:1715695113.583330  242044 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1715695113.591353  242044 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1715695113.592696  242042 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1715695113.592847  242048 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1715695113.593073  242049 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling suppor

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 871ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━